# 🧠 Panduan Lengkap Round Robin Load Balancer (2 Laptop + Web HTML)

Dokumen resmi ini menjelaskan langkah demi langkah membangun dan menjalankan sistem **Load Balancer Round Robin** menggunakan **2 laptop** — masing-masing agent menampilkan **halaman web HTML** sendiri.  

Tujuan sistem ini adalah agar saat client membuka **`http://192.168.1.72:8000/`**, halaman web akan tampil **bergantian (Round Robin)** dari dua agent berbeda.


# 1. Topologi & Tujuan

**Peran tiap laptop:**

* 💻 **Laptop 1 – Windows (192.168.1.70)**  
  → Menjalankan **Agent-1** + menampilkan **web HTML Agent-1**

* 💻 **Laptop 2 – Ubuntu (192.168.1.72)**  
  → Menjalankan:
  * **Agent-2** + web HTML Agent-2 (port 5002)  
  * **Load Balancer** (port 8000)  
  * **Client uji beban** (script Python)  
  * **Analisis log**

**Alur sistem:**

```
Client (browser/script) → http://192.168.1.72:8000/ (Load Balancer)
                         ↳ http://192.168.1.70:5001/ (Agent-1 HTML)
                         ↳ http://192.168.1.72:5002/ (Agent-2 HTML)
```


# 2. Persiapan Dasar

## 2.1. Cek IP Address

### 🖥️ Di Windows (Laptop 1)
```bat
ipconfig
```
→ Catat **IPv4 Address**, misal: `192.168.1.70`

### 💻 Di Ubuntu (Laptop 2)
```bash
ip a
```
→ Catat IP aktif, misal: `192.168.1.72`

> Jika berbeda, ubah di bagian kode `BACKENDS` pada `load_balancer.py`.


## 2.2. Instalasi Python & Flask

### 💻 Ubuntu (Laptop 2)
```bash
sudo apt update
sudo apt install python3.12 python3.12-venv -y

mkdir ~/roundrobin-lb
cd ~/roundrobin-lb
python3 -m venv venv
source venv/bin/activate
pip install flask requests
```

### 🖥️ Windows (Laptop 1)
```bat
python -m pip install --upgrade pip
python -m pip install flask
```


# 3. Konfigurasi Agent-1 (Windows – 192.168.1.70:5001)

## 3.1. Struktur Folder
```
C:\Users\HP 14S\Desktop\roundrobin-agent1\
│
├── agent1.py
└── templates\
    └── agent1.html
```


In [ ]:
# agent1.py
from flask import Flask, jsonify, request, render_template
import time, socket

app = Flask(__name__, template_folder="templates")
SERVER_ID = "Agent-1"

@app.route("/")
def home():
    return render_template("agent1.html", server_id=SERVER_ID)

@app.route("/process", methods=["GET"])
def process():
    start = time.time()
    time.sleep(0.05)
    duration = (time.time() - start) * 1000
    return jsonify({
        "server_id": SERVER_ID,
        "message": f"Request diproses oleh {SERVER_ID}",
        "host": socket.gethostname(),
        "processing_time_ms": duration,
        "client_from_lb": request.remote_addr
    })

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5001)


### `templates/agent1.html`
```html
<!DOCTYPE html>
<html lang="id">
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>{{ server_id }} - Load Balancing Demo</title>
  <style>
    body {
      font-family: Arial, sans-serif;
      background: linear-gradient(135deg, #1e3a8a, #1d4ed8);
      color: #fff; text-align: center; padding: 50px;
    }
    .card {
      background: rgba(255,255,255,0.15);
      padding: 30px; border-radius: 20px;
      display: inline-block; width: 400px;
      backdrop-filter: blur(10px);
    }
    h1 { color: #fbbf24; }
    a {
      color: #fff; background: #3b82f6;
      padding: 10px 20px; border-radius: 10px;
      text-decoration: none; margin-top: 10px; display: inline-block;
    }
    a:hover { background: #2563eb; }
  </style>
</head>
<body>
  <div class="card">
    <h1>{{ server_id }}</h1>
    <p>Backend Server aktif: Windows Agent-1</p>
    <p>Alamat: <b>http://192.168.1.70:5001</b></p>
    <p>Server ini digunakan dalam pengujian <i>Round Robin Load Balancing</i>.</p>
    <a href="/process" target="_blank">Coba Endpoint /process</a>
  </div>
</body>
</html>
```

### Jalankan Agent-1
```bat
cd C:\Users\HP 14S\Desktop\roundrobin-agent1
python agent1.py
```
Buka di browser:  
➡️ `http://192.168.1.70:5001/`  
➡️ `http://192.168.1.70:5001/process`


# 4. Konfigurasi Agent-2 (Ubuntu – 192.168.1.72:5002)
Struktur:
```
~/roundrobin-lb/
├── agent2.py
└── templates/
    └── agent2.html
```


In [ ]:
# agent2.py
from flask import Flask, jsonify, request, render_template
import time, socket

app = Flask(__name__, template_folder="templates")
SERVER_ID = "Agent-2-Ubuntu"

@app.route("/")
def home():
    return render_template("agent2.html", server_id=SERVER_ID)

@app.route("/process", methods=["GET"])
def process():
    start = time.time()
    time.sleep(0.08)
    duration = (time.time() - start) * 1000
    return jsonify({
        "server_id": SERVER_ID,
        "message": f"Request diproses oleh {SERVER_ID}",
        "host": socket.gethostname(),
        "processing_time_ms": duration,
        "client_from_lb": request.remote_addr
    })

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5002)


### `templates/agent2.html`
```html
<!DOCTYPE html>
<html lang="id">
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>{{ server_id }} - Load Balancing Demo</title>
  <style>
    body {
      font-family: Arial, sans-serif;
      background: linear-gradient(135deg, #14532d, #16a34a);
      color: #fff; text-align: center; padding: 50px;
    }
    .card {
      background: rgba(255,255,255,0.1);
      padding: 30px; border-radius: 20px;
      display: inline-block; width: 400px;
      backdrop-filter: blur(10px);
    }
    h1 { color: #86efac; }
    a {
      color: #fff; background: #22c55e;
      padding: 10px 20px; border-radius: 10px;
      text-decoration: none; margin-top: 10px; display: inline-block;
    }
    a:hover { background: #16a34a; }
  </style>
</head>
<body>
  <div class="card">
    <h1>{{ server_id }}</h1>
    <p>Backend Server aktif: Ubuntu Agent-2</p>
    <p>Alamat: <b>http://192.168.1.72:5002</b></p>
    <p>Server ini digunakan dalam pengujian <i>Round Robin Load Balancing</i>.</p>
    <a href="/process" target="_blank">Coba Endpoint /process</a>
  </div>
</body>
</html>
```

### Jalankan Agent-2
```bash
cd ~/roundrobin-lb
source venv/bin/activate
python agent2.py
```


# 5. Konfigurasi Load Balancer (Ubuntu – 192.168.1.72:8000)


In [ ]:
# load_balancer.py
from flask import Flask, jsonify, request
import requests, time, csv, os

app = Flask(__name__)
BACKENDS = ["http://192.168.1.70:5001", "http://192.168.1.72:5002"]
LOG_FILE = "lb_log.csv"
index = 0

if not os.path.exists(LOG_FILE):
    with open(LOG_FILE, "w", newline="") as f:
        csv.writer(f).writerow([
            "timestamp","client_ip","backend_url","backend_server_id",
            "backend_response_time_ms","total_response_time_ms","status_code"
        ])

@app.route("/", methods=["GET"])
def web_round_robin():
    global index
    backend_url = BACKENDS[index]
    index = (index + 1) % len(BACKENDS)
    try:
        resp = requests.get(backend_url, timeout=5)
        return resp.text, resp.status_code, {"Content-Type": resp.headers.get("Content-Type","text/html")}
    except Exception as e:
        return f"Gagal memuat halaman dari {backend_url}: {e}", 500

@app.route("/api/request", methods=["GET"])
def handle_request():
    global index
    backend_base = BACKENDS[index]
    backend_url = backend_base + "/process"
    index = (index + 1) % len(BACKENDS)
    client_ip = request.remote_addr
    start_total = time.time()

    try:
        start_backend = time.time()
        resp = requests.get(backend_url, timeout=5)
        backend_elapsed = (time.time() - start_backend) * 1000
        total_elapsed = (time.time() - start_total) * 1000
        data = resp.json()
        server_id = data.get("server_id","unknown")
        with open(LOG_FILE,"a",newline="") as f:
            csv.writer(f).writerow([
                time.strftime("%Y-%m-%d %H:%M:%S"),client_ip,backend_base,
                server_id,f"{backend_elapsed:.2f}",f"{total_elapsed:.2f}",resp.status_code
            ])
        return jsonify({
            "client_ip": client_ip,"backend_url": backend_base,
            "backend_server_id": server_id,
            "backend_processing_time_ms": backend_elapsed,
            "total_response_time_ms": total_elapsed,"status_code": resp.status_code})
    except Exception as e:
        total_elapsed = (time.time() - start_total) * 1000
        return jsonify({"error": str(e),"backend_url": backend_url,
                        "total_response_time_ms": total_elapsed}),500

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=8000)


# 6. Client Uji Beban (Ubuntu)


In [ ]:
# client_load_test.py
import requests, time, statistics

LB_URL = "http://192.168.1.72:8000/api/request"

def run_scenario(total_requests):
    print(f"\n=== Uji Beban {total_requests} Request ===")
    times, backend_counts = [], {}
    for i in range(1, total_requests + 1):
        start = time.time()
        r = requests.get(LB_URL)
        elapsed = (time.time() - start) * 1000
        try:
            data = r.json()
            backend = data.get("backend_server_id","Unknown")
        except:
            backend = "Unknown"
        times.append(elapsed)
        backend_counts[backend] = backend_counts.get(backend,0)+1
        print(f"Req-{i:03d} | {elapsed:7.2f} ms | Backend: {backend}")
    avg = sum(times)/len(times)
    std = statistics.stdev(times) if len(times)>1 else 0
    print(f"\nRata-rata: {avg:.2f} ms | Deviasi: {std:.2f} ms")
    print("Distribusi Backend:")
    for b,c in backend_counts.items():
        print(f"  - {b}: {c} request")

if __name__ == "__main__":
    for n in [50,100,200]:
        run_scenario(n)
        print("\n"+"="*40+"\n")


# 7. Analisis Log (Ubuntu)


In [ ]:
# analyze_log.py
import csv, sys, statistics

def analyze_log(filename):
    times, backend_times, status_counts = [], {}, {}
    with open(filename,newline="") as f:
        reader = csv.DictReader(f)
        for row in reader:
            try:
                t=float(row["total_response_time_ms"].replace(",","."))
            except:
                continue
            times.append(t)
            backend=row.get("backend_server_id","Unknown")
            backend_times.setdefault(backend,[]).append(t)
            status=row.get("status_code","unknown")
            status_counts[status]=status_counts.get(status,0)+1
    if not times:
        print("Tidak ada data valid di log.")
        return
    avg=sum(times)/len(times)
    std=statistics.stdev(times) if len(times)>1 else 0
    print(f"\n=== Analisis {filename} ===")
    print(f"Total entri   : {len(times)}")
    print(f"Rata-rata     : {avg:.2f} ms")
    print(f"Deviasi Std   : {std:.2f} ms\n")
    print("Per Backend:")
    for backend,lst in backend_times.items():
        b_avg=sum(lst)/len(lst)
        b_std=statistics.stdev(lst) if len(lst)>1 else 0
        print(f"- {backend}: {len(lst)} req, rata-rata {b_avg:.2f} ms, deviasi {b_std:.2f} ms")

if __name__=="__main__":
    fname=sys.argv[1] if len(sys.argv)>1 else "lb_log.csv"
    analyze_log(fname)


### Jalankan Analisis
```bash
(venv) python analyze_log.py lb_log.csv
```

Menampilkan:
- Statistik rata-rata waktu respons per backend.
- Deviasi standar dan distribusi request antar server.


# ✅ 8. Hasil & Verifikasi

Buka di browser:
```
http://192.168.1.72:8000/
```
→ Halaman **Agent-1 (biru)** dan **Agent-2 (hijau)** akan tampil **bergantian** setiap refresh.

📊 Jalankan uji beban:
```bash
(venv) python client_load_test.py
```
📈 Analisis hasil:
```bash
(venv) python analyze_log.py lb_log.csv
```
Semua hasil pengujian tersimpan di `lb_log.csv` dan siap digunakan untuk laporan penelitian.
